In [1]:
import os
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException

In [2]:
def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 20)
    return driver, wait

def save_last_index(index):
    with open("last_index.txt", "w") as file:
        file.write(str(index))

def load_last_index():
    if os.path.exists("last_index.txt"):
        with open("last_index.txt", "r") as file:
            try:
                return int(file.read().strip())
            except ValueError:
                return 0
    return 0

driver, wait = setup_driver()
driver.get("https://web.telegram.org/")
time.sleep(random.uniform(1,5))
input("لطفاً وارد تلگرام شوید و کلید Enter را فشار دهید...")

def open_group(index=0):
    try:
        chat_list = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.chatlist-chat')))
        chat_list[index].click()
        print(f"✅ گروه {index + 1} باز شد!")
        time.sleep(random.uniform(1, 3))
        return True
    except Exception as e:
        print(f"❌ خطا در باز کردن گروه: {e}")
        return False

def open_group_info():
    try:
        chat_info_element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.chat-info .person')))
        chat_info_element.click()
        print("✅ اطلاعات گروه باز شد!")
        time.sleep(random.uniform(3, 6))
        return True
    except Exception as e:
        print(f"❌ خطا در باز کردن اطلاعات گروه: {e}")
        return False

def scroll_group_members():
    try:
        members_container = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.search-super-content-container')))
        last_height = driver.execute_script("return arguments[0].scrollHeight", members_container)
        
        while True:
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", members_container)
            time.sleep(random.uniform(1, 3))
            new_height = driver.execute_script("return arguments[0].scrollHeight", members_container)
            
            if new_height == last_height:
                break  # اگر اسکرول تغییری نکرد، یعنی اعضای جدیدی لود نشده‌اند
            last_height = new_height
            print("🔄 اسکرول انجام شد و اعضای جدید بارگذاری شدند.")
    except Exception as e:
        print(f"❌ خطا در اسکرول: {e}")

def get_chat_members():
    scroll_group_members()  # اضافه کردن اسکرول برای بارگذاری اعضای جدید
    try:
        members = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.search-super-content-container .row-clickable')))
        return members[2:] if len(members) > 2 else []
    except Exception as e:
        print(f"❌ خطا در دریافت اعضا: {e}")
        return []

def click_on_button():
    try:
        time.sleep(3)
        buttons = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'chat-utils')]//button[contains(@class, 'btn-icon') and contains(@class, 'rp') and contains(@class, 'btn-menu-toggle')]")))
        
        if buttons:
            button = buttons[-1]
            if button.is_displayed():
                button.click()
                print("✅ دکمه موردنظر کلیک شد!")
                time.sleep(random.uniform(1, 3))
                
                menu_items = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'btn-menu-item') and contains(@class, 'rp-overflow')]")))
                
                for item in menu_items:
                    try:
                        text_span = item.find_element(By.XPATH, ".//span[contains(@class, 'btn-menu-item-text')]")
                        if text_span.text.strip() == "Add to contacts":
                            item.click()
                            print("✅ گزینه 'Add to contacts' انتخاب شد!")
                            time.sleep(random.uniform(1, 3))
                            return
                    except Exception:
                        continue
                
                print("❌ گزینه 'Add to contacts' در منو یافت نشد!")
            else:
                print("❌ دکمه مخفی است و امکان کلیک وجود ندارد!")
        else:
            print("❌ دکمه‌ای پیدا نشد!")
    except Exception as e:
        print(f"❌ خطا در کلیک روی دکمه: {e}")

def go_back_to_main_chat():
    try:
        driver.back()
        time.sleep(random.uniform(1, 4))
        driver.back()
        time.sleep(random.uniform(1, 3))
        print("✅ بازگشت به صفحه چت‌ها انجام شد!")
        return True
    except Exception as e:
        print(f"❌ خطا در بازگشت: {e}")
        return False

def click_members():
    global last_index
    last_index = load_last_index()
    print(f"📌 شروع پردازش از عضو شماره: {last_index}")

    while True:
        if not open_group():
            continue
        if not open_group_info():
            continue
        while True:
            members = get_chat_members()
            if not members:
                print("✅ همه اعضا بررسی شدند! وقفه کوتاه...")
                time.sleep(random.uniform(10, 20))
                break
            actions = ActionChains(driver)
            for i in range(last_index, len(members)):
                try:
                    actions.move_to_element(members[i]).perform()
                    time.sleep(random.uniform(1, 4))
                    members[i].click()
                    print(f"✅ عضو {i + 1} کلیک شد")
                    time.sleep(random.uniform(1,3))
                    click_on_button()
                    last_index = i + 1
                    save_last_index(last_index)  # ذخیره آخرین ایندکس پردازش شده
                    if not go_back_to_main_chat():
                        break
                    if not open_group_info():
                        break
                except Exception as e:
                    print(f"❌ خطا در کلیک روی عضو {i + 1}: {e}")
                    if "stale element reference" in str(e).lower():
                        refresh_page()
                        break
            time.sleep(random.uniform(1, 4))

click_members()


لطفاً وارد تلگرام شوید و کلید Enter را فشار دهید... 


📌 شروع پردازش از عضو شماره: 2119
✅ گروه 1 باز شد!
✅ اطلاعات گروه باز شد!
✅ همه اعضا بررسی شدند! وقفه کوتاه...
❌ خطا در باز کردن گروه: Message: element not interactable
  (Session info: chrome=134.0.6998.36)
Stacktrace:
	GetHandleVerifier [0x00007FF6602BFE45+26629]
	(No symbol) [0x00007FF660226010]
	(No symbol) [0x00007FF6600B914C]
	(No symbol) [0x00007FF660110D24]
	(No symbol) [0x00007FF6601028A4]
	(No symbol) [0x00007FF660137AAA]
	(No symbol) [0x00007FF660102156]
	(No symbol) [0x00007FF660137CC0]
	(No symbol) [0x00007FF660160169]
	(No symbol) [0x00007FF660137883]
	(No symbol) [0x00007FF660100550]
	(No symbol) [0x00007FF660101803]
	GetHandleVerifier [0x00007FF6606172BD+3529853]
	GetHandleVerifier [0x00007FF66062DA22+3621858]
	GetHandleVerifier [0x00007FF6606224D3+3575443]
	GetHandleVerifier [0x00007FF66038B77A+860474]
	(No symbol) [0x00007FF66023088F]
	(No symbol) [0x00007FF66022CBC4]
	(No symbol) [0x00007FF66022CD66]
	(No symbol) [0x00007FF66021C2C9]
	BaseThreadInitThunk [0x00007FFF50

KeyboardInterrupt: 

In [ ]:
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# تنظیمات مرورگر و راه‌اندازی آن
def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    return driver, WebDriverWait(driver, 2)

# باز کردن گروه
def open_group():
    try:
        chat_list = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.chatlist-chat')))
        chat_list[0].click()  # انتخاب گروه اول
        print("✅ گروه اول باز شد!")
        time.sleep(random.uniform(2, 10))
        return True
    except Exception as e:
        print(f"❌ خطا در باز کردن گروه اول: {e}")
        return False

# باز کردن اطلاعات گروه
def open_group_info():
    try:
        chat_info_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.chat-info .person')))
        chat_info_element.click()
        print("✅ اطلاعات گروه باز شد!")
        time.sleep(random.uniform(3, 6))
        return True
    except Exception as e:
        print(f"❌ خطا در باز کردن اطلاعات گروه: {e}")
        return False

# کلیک بر روی دکمه افزودن
def click_add_button():
    try:
        add_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'btn-circle') and contains(@class, 'btn-corner')]//span[contains(@class, 'tgico')]")))
        add_button.click()
        print("✅ دکمه افزودن کلیک شد!")
        time.sleep(random.uniform(2, 5))
    except Exception as e:
        print(f"❌ خطا در کلیک روی دکمه افزودن: {e}")

# کلیک روی همه اعضای <a> در <ul class="chatlist">
def click_all_members():
    try:
        members = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[@class='sidebar-left-section-content selector-list-section-content']//ul[@class='chatlist']/a")))

        for index, member in enumerate(members):
            wait.until(EC.element_to_be_clickable(member))  # اطمینان از اینکه عنصر قابل کلیک است
            member.location_once_scrolled_into_view  # اسکرول به محل عضو
            time.sleep(0.5)  # تأخیر کوتاه
            member.click()
            print(f"✅ عضو {index + 1} کلیک شد!")
            time.sleep(random.uniform(1, 2))  # تأخیر تصادفی بین کلیک‌ها

        print("✅ همه اعضا کلیک شدند!")
    except Exception as e:
        print(f"❌ خطا در کلیک روی اعضا: {e}")

# کلیک روی دکمه تایید
def confirm_addition():
    try:
        print("⏳ در حال جستجوی دکمه تایید...")
        
        add_confirm_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'add-members-container') and contains(@class, 'active')]//button[contains(@class, 'btn-circle') and contains(@class, 'btn-corner') and contains(@class, 'is-visible')]")))
        
        # اطمینان از اینکه دکمه قابل مشاهده و فعال است
        if add_confirm_button.is_displayed() and add_confirm_button.is_enabled():
            add_confirm_button.click()  # کلیک روی دکمه
            print("✅ دکمه تایید کلیک شد!")
            time.sleep(random.uniform(1, 7))
        else:
            print("🚨 دکمه تایید در دسترس نیست!")

    except Exception as e:
        print(f"❌ خطا در کلیک روی دکمه تایید: {e}")

# شروع فرآیند
driver, wait = setup_driver()
driver.get("https://web.telegram.org/")
time.sleep(random.uniform(5, 10))

# اجرای مراحل به ترتیب
if open_group() and open_group_info():
    click_add_button()
    click_all_members()
    confirm_addition()  # کلیک روی دکمه تایید

print("✅ عملیات به پایان رسید!")
